# ERA5 Correction via MintPy

In [ ]:
# can try mintpy approach
#!smallbaselineApp.py FernandinaSenDT128.txt --dostep correct_troposphere
# can try barebones pyaps?
#import PyAPS
#!tropo_pyaps3.py -d 20151002 20151003 --hour 12 -m ERA5 -g inputs/geometryRadar.h5

In [ ]:
#https://github.com/ASFHyP3/hyp3-docs/blob/develop/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb

In [1]:
import mintpy
from pathlib import Path
from dateutil.parser import parse as parse_date
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from pathlib import Path
from typing import List, Union
import os
import glob
import rasterio
import matplotlib.pyplot as plt
from mintpy.utils import ptime, readfile, writefile, utils as ut

In [2]:
# project_name = 'data'
# data_dir = Path.cwd() / project_name
# #work_dir = Path.cwd() / 'training/data'

In [3]:
#data_dir.mkdir(parents=True, exist_ok=True)

In [4]:
# def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
#     """Get the common overlap of  a list of GeoTIFF files
    
#     Arg:
#         file_list: a list of GeoTIFF files
    
#     Returns:
#          [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
#          corner coordinates of the common overlap
#     """
    
#     corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

#     ulx = max(corner['upperLeft'][0] for corner in corners)
#     uly = min(corner['upperLeft'][1] for corner in corners)
#     lrx = min(corner['lowerRight'][0] for corner in corners)
#     lry = max(corner['lowerRight'][1] for corner in corners)
#     return [ulx, uly, lrx, lry]

# def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
#     """Clip all GeoTIFF files to their common overlap
    
#     Args:
#         data_dir:
#             directory containing the GeoTIFF files to clip
#         overlap:
#             a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
#             corner coordinates of the common overlap
#     Returns: None
#     """

    
#     files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

#     for extension in files_for_mintpy:

#         for file in data_dir.rglob(f'*{extension}'):

#             dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

#             gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [5]:
#files = data_dir.glob('*VVP012*/*_dem.tif')
#overlap = get_common_overlap(files)
#clip_hyp3_products_to_common_overlap(data_dir, overlap)

In [6]:
#!smallbaselineApp.py --dir {data_dir} {mintpy_config} --dostep load_data

In [7]:
#directories = [os.path.abspath(x[0]) for x in os.walk(data_dir)]
#directories.remove(os.path.abspath(data_dir)) # If you don't want your main directory included
#directories.remove('/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/training/data/S1AA_20190505T135154_20190517T135155_VVP012_INT80_G_weF_5C8F/.ipynb_checkpoints')
weather_dir = '/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/'
directories = glob.glob('/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1*/')
#directories

In [8]:
for i in directories:
    os.chdir(i)         # Change working Directory
    !pwd
    data_dir = Path.cwd()
    mintpy_config = data_dir / 'mintpy_config.txt'
    mintpy_config.write_text(
    f"""
    mintpy.load.processor        = hyp3
    ##---------interferogram datasets:
    mintpy.load.unwFile          = *_unw_phase.tif
    mintpy.load.corFile          = *_corr.tif
    ##---------geometry datasets:
    mintpy.load.demFile          = *_dem.tif
    mintpy.load.incAngleFile     = *_lv_theta.tif
    mintpy.load.azAngleFile      = *_lv_phi.tif
    mintpy.load.waterMaskFile    = *_water_mask.tif
    """)
    !smallbaselineApp.py --dir {data_dir} {mintpy_config} --dostep load_data
    geom_path = 'inputs/geometryGeo.h5'

    tif_filename = glob.glob('*_unw_phase.tif')[0] #clipped
    unw_filename = f'{tif_filename[:-4]}.unw'

    data, atr = readfile.read(tif_filename, datasetName='phase')
    #atr['OG_FILE_PATH'] = atr['FILE_PATH']
    atr['FILE_TYPE'] = '.unw'
    writefile.write({'phase':data},unw_filename,metadata=atr)
    #!gdal_translate $tif_filename $unw_filename -of ISCE
    #print(unw_filename)
    #!pwd
    #!gdalinfo $unw_filename
    !tropo_pyaps3.py -f $unw_filename -g $geom_path --weather-dir $weather_dir

/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC
MintPy version v1.3.3, date 2022-04-14
--RUN-at-2022-06-02 17:36:35.018054--
Current directory: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Project name: mintpy_config
Go to work directory: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC
read custom template file: /mnt/working/eg

In [ ]:
unw_corr = '/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC_unw_phase_ERA5.unw'
ok=rasterio.open(unw_corr).read(2)

In [ ]:
unw = '/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/data/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC/S1BB_20210402T152635_20210520T152638_VVP048_INT80_G_weF_28FC_unw_phase.unw'
ok2=rasterio.open(unw).read(2)

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,10))
l=ax[0].imshow(ok,vmin=-10,vmax=10)
f.colorbar(l,ax=ax[0])
l1=ax[1].imshow(ok2,vmin=-10,vmax=10)
f.colorbar(l1,ax=ax[1])
l2 = ax[2].imshow(ok2-ok,vmin=-10,vmax=10)
f.colorbar(l2,ax=ax[2])